In [10]:

import time

import numpy as np
import scipy.sparse as sp
from scipy.special import expit  # logistic function

from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.utils import gen_even_slices

# https://gist.github.com/dwf/359323

In [23]:
# this is just straight from scipy...
# this is a simplified solution without
# some of the "nice" optimization

class GaussianBernoulliRBM(BaseEstimator, TransformerMixin):
    def __init__(self, n_components=256, learning_rate=0.1, batch_size=10,
                 n_iter=10, sigma=1, verbose=False):
        self.n_components = n_components
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.n_iter = n_iter
        self.verbose = verbose
        self.sigma = sigma
    
    def _mean_visibles_given_hiddens(self, h, v):
        """Conditional probability derivation P(v|h).
           P(v|h) = N( Wh + bias_vis, sigma^2)
           Page 38 (http://www.ini.rub.de/data/documents/tns/masterthesis_janmelchior.pdf)
        """
        p = (np.dot(h, self.components_)) + self.intercept_visible_

        return norm.rvs(loc=p, scale=np.square(self.sigma))
    
    def reconstruct(self, v):
        """reconstruct by computing positive phase
           followed by the negative phase
        """
        h_ = self._sample_hiddens(v)
        v_ = self._mean_visibles_given_hiddens(h_, v)
        return v_
    
    def _sigma_gradient(self, v, h):
        """
            Computes the partial derivative with
            respect to sigma
            Page 41 (http://www.ini.rub.de/data/documents/tns/masterthesis_janmelchior.pdf)
        """
        t1 = (np.square(v - self.intercept_visible_) /
              np.power(self.sigma_, 3))

        t2 = (2 * v) / np.power(self.sigma_, 3)

        t3 = safe_sparse_dot(h, self.components_)

        return check_array(t1 - (t2 * t3))
    
    def transform(self, X):     
        """P(h=1|v=X)
        """
        return self._mean_hiddens(X)
    
    def _mean_hiddens(self, v):
        """Compute P(h=1|v)
        """
        
        p = np.dot(v, self.components_.T)
        p += self.intercept_hidden_
        return expit(p, out=p)
    
    def _sample_hiddens(self, v):
        """Sample from the distribution P(h|v).
        """
        p = np.dot(h, self.components_)
        p += self.intercept_visible_
        expit(p, out=p)
        # see hinton chpt 3
        return (np.random.random_sample(size=p.shape) < p)
    
    def _sample_visibles(self, h):
        """Sample from the distribution P(v|h).
        """
        p = np.dot(h, self.components_)
        p += self.intercept_visible_
        expit(p, out=p)
        return (np.random.random_sample(size=p.shape) < p)
    
    def _free_energy(self, v):
        """Computes the free energy F(v) = - log sum_h exp(-E(v,h)).
        Parameters
        ----------
        v : array-like, shape (n_samples, n_features)
            Values of the visible layer.
        Returns
        -------
        free_energy : array-like, shape (n_samples,)
            The value of the free energy.
        """
        return (- np.dot(v, self.intercept_visible_)
                - np.logaddexp(0, np.dot(v, self.components_.T)
                               + self.intercept_hidden_).sum(axis=1))
    
    def gibbs(self, v):
        """Perform one Gibbs sampling step.
        """
        h_ = self._sample_hiddens(v)
        v_ = self._sample_visibles(h_)

        return v_
    
    def partial_fit(self, X, y=None):
        """Fit the model to the data X which should contain a partial
        segment of the data.
        """
        if not hasattr(self, 'components_'):
            self.components_ = np.asarray(
                self.random_state_.normal(
                    0,
                    0.01,
                    (self.n_components, X.shape[1])
                ),
                order='F')
        if not hasattr(self, 'intercept_hidden_'):
            self.intercept_hidden_ = np.zeros(self.n_components, )
        if not hasattr(self, 'intercept_visible_'):
            self.intercept_visible_ = np.zeros(X.shape[1], )
        if not hasattr(self, 'h_samples_'):
            self.h_samples_ = np.zeros((self.batch_size, self.n_components))

        self._fit(X, self.random_state_)
        

    def _fit(self, v_pos):
        """Inner fit for one mini-batch.
        Adjust the parameters to maximize the likelihood of v using
        Stochastic Maximum Likelihood (SML).
        """
        h_pos = self._mean_hiddens(v_pos)
        v_neg = self._sample_visibles(self.h_samples_)
        h_neg = self._mean_hiddens(v_neg)

        lr = float(self.learning_rate) / v_pos.shape[0]
        update = np.dot(v_pos.T, h_pos).T
        update -= np.dot(h_neg.T, v_neg)
        self.components_ += lr * update
        self.intercept_hidden_ += lr * (h_pos.sum(axis=0) - h_neg.sum(axis=0))
        self.intercept_visible_ += lr * (np.asarray(
                                         v_pos.sum(axis=0)).squeeze() -
                                         v_neg.sum(axis=0))

        h_neg[np.random.uniform(size=h_neg.shape) < h_neg] = 1.0  # sample binomial
        self.h_samples_ = np.floor(h_neg, h_neg)
        
    def score_samples(self, X):
        """compute pseudo-likelihood X"""
        v = X
        #v = check_array(X, accept_sparse='csr')
        #rng = check_random_state(self.random_state)

        # Randomly corrupt one feature in each sample in v.
        ind = (np.arange(v.shape[0]),
               np.random.randint(0, v.shape[1], v.shape[0]))

        v_ = v.copy()
        v_[ind] = 1 - v_[ind]

        fe = self._free_energy(v)
        fe_ = self._free_energy(v_)
        return v.shape[1] * log_logistic(fe_ - fe)
    
    
    def fit(self, X, y=None):
        """Fit the model to the data X.
        """
        n_samples = X.shape[0]
        
        self.components_ = np.random.normal(0, 0.01, (self.n_components, X.shape[1]))
        self.intercept_hidden_ = np.zeros(self.n_components, )
        self.intercept_visible_ = np.zeros(X.shape[1], )
        self.h_samples_ = np.zeros((self.batch_size, self.n_components))

        n_batches = int(np.ceil(float(n_samples) / self.batch_size))
        batch_slices = list(gen_even_slices(n_batches * self.batch_size,
                                            n_batches, n_samples))
        verbose = self.verbose
        begin = time.time()
        for iteration in range(1, self.n_iter + 1):
            for batch_slice in batch_slices:
                self._fit(X[batch_slice])

            if verbose:
                end = time.time()
                print("[%s] Iteration %d, pseudo-likelihood = %.2f,"
                      " time = %.2fs"
                      % (type(self).__name__, iteration,
                         self.score_samples(X).mean(), end - begin))
                begin = end

        return self

In [27]:
rbm = BernoulliRBM(100)

In [28]:
X = np.random.randint(low=0, high=1, size=(100000, 100))
rbm.fit(X)

BernoulliRBM(batch_size=10, learning_rate=0.1, n_components=100, n_iter=10,
       verbose=False)

In [29]:
rbm.transform(X)

array([[ 0.68772362,  0.68763322,  0.68757298, ...,  0.6871598 ,
         0.68761404,  0.68850964],
       [ 0.68772362,  0.68763322,  0.68757298, ...,  0.6871598 ,
         0.68761404,  0.68850964],
       [ 0.68772362,  0.68763322,  0.68757298, ...,  0.6871598 ,
         0.68761404,  0.68850964],
       ..., 
       [ 0.68772362,  0.68763322,  0.68757298, ...,  0.6871598 ,
         0.68761404,  0.68850964],
       [ 0.68772362,  0.68763322,  0.68757298, ...,  0.6871598 ,
         0.68761404,  0.68850964],
       [ 0.68772362,  0.68763322,  0.68757298, ...,  0.6871598 ,
         0.68761404,  0.68850964]])